In [1]:
#首先把需要用的库引入进来
import requests
from IPython.display import Image
import time
from hashlib import md5
import json
import os
import marvel
import sys
from multiprocessing.pool import Pool
from functools import partial

In [2]:
#获取Marvel所有英雄的基本数据（API）
#http://gateway.marvel.com/v1/public/comics?ts=1&apikey=1234&hash=ffd275c5130566a2916217b101f26150
#public key: b429b36ad9c859671f28487a8acdca22 
#private key: 10486b67a96262c7c4859f963ae11c36d94352e6

marvel.pbkey = '799bc03dd2d54122769693bdb347f571'
marvel.pvkey = 'b72ea3902488bafd62479523874582581b82e122'


#定义md5key生成函数
# def hashkey(pvkey,pbkey):
#     ts = str(time.time())[-6:]   #timestamp ts
#     key = ts + pvkey + pbkey
#     _md5key = md5(key.encode('utf-8')).hexdigest()
#     return _md5key,ts

In [10]:

#获取指定类别名的数据总条目数，返回整数:total
def get_total(data_class):
    if type(data_class) != str:
        raise  Exception("func 'get_total()' need str type")
            
    limit = '1'
    offset = 0
    md5key,ts = marvel.hashkey()
    url =  "https://gateway.marvel.com:443/v1/public/"+data_class+"?"\
            "offset="+str(offset)+"&limit="+limit+"&ts="+ts+"&apikey="+marvel.pbkey+"&hash="+md5key
    contents = requests.get(url,timeout = None)
    if contents.status_code == 200:
            print('Get',data_class,'total.......[Done]')
    else:
            print('Get',data_class,'total.......[Fail]',contents.status_code)
            sys.exit()
    contents_r = json.loads(contents.text)
    total = contents_r['data']['total']
    print('total',data_class,':',total)
    return total



#total = get_total(data_class)

#单独抓取一次的函数，参数为类别名:data_class,偏移量：offset
def get_data(data_class, offset = 0):
    if type(data_class) != str:
        raise Exception("func 'get_data()' need str type")
    count = 0
    item_list =[]
    #offset = 0
    limit = '100'
    
    while True :
        md5key,ts = marvel.hashkey()
        url =  "https://gateway.marvel.com:443/v1/public/"+data_class+"?"\
                          "offset="+str(offset)+"&limit="+limit+"&ts="+ts+"&apikey="+marvel.pbkey+"&hash="+md5key
        #print(url)
        print('['+time.ctime()+']',offset,'-',offset+100,'fetching...............',end = '')
        contents = requests.get(url, timeout = None)
        
        if contents.status_code == 200:
            print('[Done]')
            #fail_locate = '' #增加出错位置标记
        else:
            print('[Fail]',contents.status_code)
            #fail_locate = offset #增加出错位置标记
            time.sleep(1)
            break
        print('['+time.ctime()+']',offset,'-',offset+100,'Decoding...............',end='')
        contents_r = json.loads(contents.text)
        print('[Done]')
        #count += contents_r['data']['count']
        #print(count)
        #offset += 100
        print('['+time.ctime()+']',offset,'-',offset+100,'Appending to Dict......',end='')

        for item in contents_r['data']['results']:
            item_list.append(item)     
        print('[Done]')

        return item_list
        time.sleep(1)
        break
        #count -= 1

#def fail(locatioin):
    
        

#多进程抓取指定data_class类别的数据，每个进程100条数据
#参数data_class 为marvel类别保留字，hyper为开启进程数量
#本函数自动获取类别名下数据总条数:total，然后自动设置偏移量:offset完成全部抓取
#返回两个值，list类型的results_list和整数型的total(总条目数)

def hyper_get_data(data_class,hyper=1):
    data_class_i = data_class
    total = get_total(data_class_i)
    total_left = total
    #生成offset_list
    offset_list = [0]
    while total_left >= 0:
        offset_list.append(offset_list[-1]+100)
        total_left -= 100
    offset_list = offset_list[:-1]
    
    t=time.time()
    pool = Pool(hyper)
    get_data_p = partial(get_data, data_class_i)
    results = pool.map(get_data_p,offset_list) #返回结果列表和出错列表
    pool.close()
    pool.join()
    print('Done!\n',len(results),'pages got!')
    print(round(time.time()-t,2),'sec spended.')
    print('Coverting list......', end = '')
    results_list = []
    for result in results:
        results_list += result
    print('Done!','\n',len(results_list),data_class,'got!')
    
    return results_list,total
    
#将抓取完成的类别数据写入相应的json文件，文件名为类别名。
def write_file(data_class,total,res):        
    item_dict={'data_class':data_class,'total':total,'results':res}
    #print(item_dict)
    print('['+time.ctime()+']','Converting to JSON ......',end='')
    item_j = json.dumps(item_dict)
    print('[Done]')
    print('['+time.ctime()+']','Writting to',os.getcwd()+'/'+data_class+'.json ......',end='')
    with open(data_class+'.json','w') as item_f:
        item_f.write(str(item_j))
        print('[Done]')
       

In [11]:
class_name = ['characters','comics','creators','events','series','stories']
# data_class = class_name[0]
# res,total=hyper_get_data(data_class,15)


In [12]:
# write_file(data_class,total,res)

In [14]:
data_class = class_name[1]
res,total=hyper_get_data(data_class,20)
#write_file(data_class,total,res)

Get comics total.......[Fail] 429


SystemExit: 

In [ ]:
#write_file(data_class,total,res)